In [1]:
from pathlib import Path
from itertools import cycle, count

import yfinance as yf
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from scipy import stats
from plotly.subplots import make_subplots

In [2]:
import plotly.io as pio

# for vscode
pio.renderers.default = "browser"

In [3]:
# Load S&P 500 data
csv_file = Path("sp500.csv")
if not csv_file.exists():
    df = yf.download("^GSPC")

    df = df.reset_index()  # Use an integer index
    df.columns = df.columns.droplevel(1)  # Remove the second level (ticker)
    df.to_csv(csv_file)
else:
    df = pd.read_csv(csv_file)

df = df.rename(columns={"Date": "date", "Close": "value"})
df = df[["date", "value"]]
df["date"] = pd.to_datetime(df["date"])

df

,date,value
0,1927-12-30,17.660000
1,1928-01-03,17.760000
2,1928-01-04,17.719999
3,1928-01-05,17.549999
4,1928-01-06,17.660000
...,...,...
24445,2025-04-28,5528.750000
24446,2025-04-29,5560.830078
24447,2025-04-30,5569.060059
24448,2025-05-01,5604.140137


In [4]:
# All drawdowns over threshold
threshold = -0.10

df["cummax"] = df["value"].cummax()
df["start"] = df.groupby("cummax")["date"].transform("first")
df["end"] = df.groupby("cummax")["date"].transform("last")

min_indices = df.groupby("cummax")["value"].transform("idxmin")
df["bottom"] = df.loc[min_indices, "date"].values
df["drawdown"] = (df.groupby("cummax")["value"].transform("min") - df["cummax"]) / df["cummax"]

drawdowns = df[df["drawdown"] < threshold].groupby("cummax").max().reset_index()
drawdowns = drawdowns[["start", "bottom", "end", "drawdown"]]

drawdowns["to_bottom"] = drawdowns["bottom"] - drawdowns["start"]
drawdowns["to_recovery"] = drawdowns["end"] - drawdowns["start"]

drawdowns.describe()

,start,bottom,end,drawdown,to_bottom,to_recovery
count,26,26,26,26.000000,26,26
mean,1985-07-10 17:32:18.461538432,1986-04-15 15:41:32.307692288,1988-02-23 12:00:00,-0.254889,278 days 22:09:13.846153848,957 days 18:27:41.538461536
min,1928-05-14 00:00:00,1928-06-12 00:00:00,1928-08-27 00:00:00,-0.861896,13 days 00:00:00,49 days 00:00:00
25%,1966-07-07 06:00:00,1967-02-12 18:00:00,1967-07-31 18:00:00,-0.321255,57 days 18:00:00,187 days 06:00:00
50%,1988-09-16 00:00:00,1989-01-01 00:00:00,1989-12-24 00:00:00,-0.198485,179 days 00:00:00,431 days 00:00:00
75%,2005-11-19 06:00:00,2007-08-01 06:00:00,2011-10-11 18:00:00,-0.125634,448 days 06:00:00,734 days 18:00:00
max,2025-02-19 00:00:00,2025-04-08 00:00:00,2025-05-02 00:00:00,-0.101137,989 days 00:00:00,9136 days 00:00:00
std,NaN,NaN,NaN,0.180043,281 days 02:13:55.076613952,1823 days 19:59:10.657426944


In [5]:
dat = drawdowns.copy()

dat["to_bottom"] = dat["to_bottom"].dt.days
dat["to_recovery"] = dat["to_recovery"].dt.days
dat["drawdown"] = dat["drawdown"].apply(lambda x: f"{x:.2%}")
dat["start"] = dat["start"].dt.strftime("%Y-%m-%d")
dat["bottom"] = dat["bottom"].dt.strftime("%Y-%m-%d")
dat["end"] = dat["end"].dt.strftime("%Y-%m-%d")

dat = dat.rename(
    columns={
        "start": "Start",
        "bottom": "Bottom",
        "end": "End",
        "drawdown": "Drawdown",
        "to_bottom": "Days To Bottom",
        "to_recovery": "Days To Recovery",
    }
)

fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=list(dat.columns), 
                align="left"
            ),
            cells=dict(
                values=[dat[col] for col in dat.columns],
                align="right"
            ),
        )
    ]
)
fig.update_layout(
    title=f"S&P 500 Drawdowns ({threshold:.0%})"
)
fig.write_html("docs/drawdowns.html", auto_open=True)

In [6]:
fig = px.scatter(
    dat.iloc[:-1],
    title="S&P 500 Recovery Time",
    x="Days To Bottom",
    y="Days To Recovery",
    log_x=True,
    log_y=True,
)
fig.add_trace(
    go.Scatter(
        x=dat.iloc[-1:]["Days To Bottom"],
        y=dat["Days To Recovery"].iloc[-1:],
        mode="markers",
        marker=dict(size=5, color="black"),
        name=f"{dat.iloc[-1]["Start"]} (so far)",
    )
)

log_x = np.log10(dat["Days To Bottom"])
log_y = np.log10(dat["Days To Recovery"])
slope, intercept, _, _, _ = stats.linregress(log_x, log_y)
x_range = np.logspace(
    np.log10(dat["Days To Bottom"].min() * 0.9),
    np.log10(dat["Days To Bottom"].max() * 1.1),
    2,
)
residuals = log_y - (slope * log_x + intercept)
channel_width = 2 * residuals.std()
fig.add_trace(
    go.Scatter(
        x=x_range,
        y=10 ** (intercept + slope * np.log10(x_range) + channel_width),
        mode="lines",
        name="Upper Channel",
        line=dict(color="black", width=0.5, dash="dash"),
    )
)
fig.add_trace(
    go.Scatter(
        x=x_range,
        y=10 ** (intercept + slope * np.log10(x_range) - channel_width),
        mode="lines",
        name="Lower Channel",
        line=dict(color="black", width=0.5, dash="dash"),
    )
)

fig.update_layout(
    xaxis_title="Days Until Market Bottom (log scale)",
    yaxis_title="Days Until Market Recovery (log scale)",
    hoverlabel_namelength=-1,
)

fig.write_html("docs/scatter.html", auto_open=True)

In [7]:
fig = make_subplots(
    rows=1,
    cols=1,
    specs=[[{"type": "scatter"}]],
    subplot_titles=("S&P 500 Drawdowns Aligned",),
    x_title="Days",
    y_title="Drawdown (%)",
)
fig.update_layout(
    hoverlabel_namelength=-1,
    yaxis_tickformat=".2%",
    margin_l=100,
)
fig.update_annotations(selector=dict(text="Drawdown (%)"), xshift=-70)

for i, drawdown, color in zip(count(0), drawdowns.iloc[:-1].iloc, cycle(px.colors.qualitative.Plotly)):
    mask = (drawdown.start <= df["date"]) & (df["date"] <= drawdown.end)
    period = df[mask].copy()

    period["i"] = (period["date"] - drawdown.start).dt.days
    period["pct"] = (period["value"] - period["value"].iloc[0]) / period["value"].iloc[0]

    legend_group = f"drawdown_{i}"
    trace_name = f"{drawdown.start.date()}, {drawdown.to_bottom.days}d bottom, {drawdown.to_recovery.days}d recovery"

    fig.add_trace(
        go.Scatter(
            x=period["i"],
            y=period["pct"],
            name=trace_name,
            mode="lines",
            line=dict(width=0.5, color=color),
            legendgroup=legend_group,  
            showlegend=True,
            customdata=[drawdown.to_recovery.days],
            visible=True if drawdown.to_recovery.days < 365 * 3 else "legendonly",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[drawdown.to_bottom.days, drawdown.to_recovery.days],
            y=[drawdown.drawdown, period["pct"].iloc[-1]],
            mode="markers",
            name=trace_name,
            marker=dict(size=4, color=color),
            legendgroup=legend_group,
            showlegend=False,
            customdata=[drawdown.to_recovery.days],
            visible=True if drawdown.to_recovery.days < 365 * 3 else "legendonly",
        )
    )


drawdown = drawdowns.iloc[-1]
mask = (drawdown.start <= df["date"]) & (df["date"] <= drawdown.end)
period = df[mask].copy()

period["i"] = (period["date"] - drawdown.start).dt.days
period["pct"] = (period["value"] - period["value"].iloc[0]) / period["value"].iloc[0]

fig.add_trace(
    go.Scatter(
        x=period["i"],
        y=period["pct"],
        name=f"{drawdown.start.date()} (so far)",
        mode="lines",
        line=dict(width=1, color="black"),
    )
)

fig.update_layout(
    updatemenus=[
        dict(
            x=1.0,
            y=1.12,
            xanchor="right",
            yanchor="top",
            pad={"r": 10, "t": 10},
            direction="down",
            buttons=list(
                [
                    dict(
                        label="Hide Longer Drawdowns (> 3 years)",
                        method="update",
                        args=[
                            {
                                "visible": [
                                    True if d.customdata[0] < 365 * 3 else "legendonly"
                                    for d in fig.data[:-1]
                                ]
                                + [True]
                            }
                        ],
                    ),
                    dict(
                        label="Hide Longer Drawdowns (> 2 years)",
                        method="update",
                        args=[
                            {
                                "visible": [
                                    True if d.customdata[0] < 365 * 2 else "legendonly"
                                    for d in fig.data[:-1]
                                ]
                                + [True]
                            }
                        ],
                    ),
                    dict(
                        label="Hide Longer Drawdowns (> 1 years)",
                        method="update",
                        args=[
                            {
                                "visible": [
                                    True if d.customdata[0] < 365 * 1 else "legendonly"
                                    for d in fig.data[:-1]
                                ]
                                + [True]
                            }
                        ],
                    ),
                    dict(
                        label="Hide All Historical",
                        method="update",
                        args=[
                            {"visible": ["legendonly"] * (len(fig.data) - 1) + [True]}
                        ],
                    ),
                    dict(
                        label="Show All Historical",
                        method="update",
                        args=[{"visible": [True] * (len(fig.data) - 1) + [True]}],
                    ),
                ]
            ),
        )
    ]
)

fig.write_html("docs/aligned.html", auto_open=True)

In [8]:
fig = make_subplots(
    rows=1,
    cols=1,
    specs=[[{"type": "scatter"}]],
    subplot_titles=(f"S&P 500 Drawdowns Highlighted ({threshold:.0%})",),
    x_title="Days",
    y_title="Value (log scale)",
)

dat = df.copy()

for drawdown in drawdowns.iloc:
    mask = (drawdown.start <= df["date"]) & (df["date"] <= drawdown.end)
    period = dat[mask]

    fig.add_trace(
        go.Scatter(
            x=period["date"],
            y=period["value"],
            mode="lines",
            line=dict(width=1, color="red"),
            name=f"{drawdown.start.date()}, {drawdown.drawdown:.2%} max, {drawdown.to_bottom.days}d bottom, {drawdown.to_recovery.days}d recovery",
            showlegend=False,
        )
    )

    dat.loc[mask.shift(1) & mask.shift(-1), "value"] = np.nan

fig.add_trace(
    go.Scatter(
        x=dat["date"],
        y=dat["value"],
        mode="lines",
        line=dict(width=1, color="blue"),
        name="S&P 500",
        showlegend=False,
    )
)

# Set log scales on both axes
fig.update_layout(
    hoverlabel_namelength=-1,
    yaxis_type="log",
    
)

fig.write_html("docs/chart.html", auto_open=True)

In [10]:
dd = drawdowns[drawdowns["to_recovery"] < pd.Timedelta(days=365 * 3)].copy()
dd["max_gain"] = (1 - dd["drawdown"]) / (1 + dd["drawdown"]) - 1
dd["bottom_to_recovery"] = dd["end"] - dd["bottom"]

dd["apy"] = (
    (1 + dd["max_gain"]) ** (365 / dd["bottom_to_recovery"].dt.days) - 1
)

dd#.describe()

,start,bottom,end,drawdown,to_bottom,to_recovery,max_gain,bottom_to_recovery,apy
0,1928-05-14,1928-06-12,1928-08-27,-0.102740,29 days,105 days,0.229008,76 days,1.692141
2,1955-09-23,1955-10-11,1955-11-11,-0.105851,18 days,49 days,0.236765,31 days,11.206928
3,1956-08-02,1957-10-22,1958-09-23,-0.214746,446 days,782 days,0.546947,336 days,0.606309
4,1959-08-03,1960-10-25,1961-01-26,-0.140175,449 days,542 days,0.326054,93 days,2.027048
5,1961-12-12,1962-06-26,1963-08-30,-0.279736,196 days,626 days,0.776758,430 days,0.628898
6,1966-02-09,1966-10-07,1967-05-03,-0.221773,240 days,448 days,0.569945,208 days,1.206682
7,1967-09-25,1968-03-05,1968-04-26,-0.101137,162 days,214 days,0.225034,52 days,3.156544
10,1980-11-28,1982-08-12,1982-11-02,-0.271136,622 days,704 days,0.743995,82 days,10.889713
11,1983-10-10,1984-07-24,1985-01-18,-0.143817,288 days,466 days,0.335949,178 days,0.811089
12,1987-08-25,1987-12-04,1989-07-25,-0.335095,101 days,700 days,1.007949,599 days,0.529263
